In [1]:
import cv2
import deepface

from tqdm.notebook import tqdm

import numpy as np
from deepface.extendedmodels import Race
from tensorflow.keras.preprocessing import image

import mtcnn

In [2]:
face_min_height_scale = 5

In [3]:
race_labels = ['asian', 'indian', 'black', 'white', 'middle eastern', 'latino hispanic']

In [4]:
model = Race.loadModel()

In [5]:
cap = cv2.VideoCapture('video1.mp4')

In [6]:
fps = cap.get(cv2.CAP_PROP_FPS)
fps = round(fps)
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

face_min_height = int(height//face_min_height_scale)

In [7]:
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter('video1_out3.mp4', fourcc, fps, (width, height))

In [8]:
face_detector = mtcnn.MTCNN()

def face_detect_mtcnn(img):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    detections = face_detector.detect_faces(img_rgb)

    faces = []
    if len(detections) > 0:
        for d in detections:        
            x, y, w, h = d["box"]
            faces.append((x,y,w,h))
            
    return faces

In [9]:
def ethnicity_detect(img, target_size=(224, 224)):
    result = 'Unknown'
    
    try:
        img = cv2.resize(img, target_size)
        img_pixels = image.img_to_array(img)
        img_pixels = np.expand_dims(img_pixels, axis = 0)
        img_pixels /= 255

        preds = model.predict(img_pixels)
        result = race_labels[np.argmax(preds)]
        
    except:
        pass
    
    return result
    

In [10]:
def face_detect(img):
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray,
                                          scaleFactor = 1.3,
                                          minNeighbors = 5,
                                          minSize = (face_min_height, face_min_height))
    return faces
    

In [11]:
def show_faces(img, faces):
    for (x,y,w,h) in faces:
        img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        
        face = img[y:y+h, x:x+w]
        
        race_label = ethnicity_detect(face)
        img = cv2.putText(img, race_label, (int(x),int(y+h)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        
    return img

def cut_faces(img, faces):
    imgs = []
    for (x,y,w,h) in faces:
        imgs.append(img[y:y+h, x:x+w])
    return imgs

In [ ]:
count = 0
frame_face_count = 0

pbar = tqdm(total=length)


while(cap.isOpened()):
    cap.set(1, count)
    ret, frame = cap.read()
    
    if ret:
        
        faces = face_detect_mtcnn(frame)

        frame = show_faces(frame, faces)

        out.write(frame)

        count+=1
        pbar.update()

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    else:
        break
        
cap.release()
out.release()
pbar.close()

  0%|          | 0/61154 [00:00<?, ?it/s]